# PRiSM Tool

This notebook shows how to define and run a 2D fluid simulation using a non-uniform mesh, and extract signals from advected particles. 

It was first developed for a gathering with [PRiSM](https://www.rncm.ac.uk/research/research-centres-rncm/prism/) collaborators during and following the gather in Manchester, Sept 23-24, 2021.

It is based on code used to explore new discretizations for the stationary [Stokes equations][1].
These equations describe fluid flow in which viscous forces dominate, and are used to model
microfluidics as well as the slow creep of solids, in particular rock in Earth's mantle and
ice sheets. The equations arise by balancing viscous forces (forces arising from frictional resistance to velocity gradients in fluids) and pressure forces with applied forces (e.g. gravity), and enforcing conservation of mass, here by enforcing that the fluid is incompressible. 

The central technical challenge is solving these equations on a non-uniform yet highly-structured
computational mesh. The computational domain (here a square) is divided into subdomains of
regular shape but which may be subdivided to allow for greater accuracy in certain parts of the domain.

The underlying assumption is that because there is an elegance and inherent aesthetic value in physical processes, it's worth while to use simulations of these processes in the composition of music.

Here, a simple and direct approach is proposed to allow quick "mapping" of these simulations to simple parameters streams which may be of use in composition. This is of course a dangerous idea in many ways, as the underlying elegance and physical analogy may easily be lost. Nevertheless, this is presented as starting point for further explorations of this system and its aesthetic interpretation.

A key computational method is particle advection. Point particles are pushed along by a given velocity field. This provides a time-parameterized series of data which can be used in the composition of musical works (which can in some sense also be though of as time-parameterized series of data).


[1]: https://en.wikipedia.org/wiki/Stokes_flow
[2]: https://en.wikipedia.org/wiki/Stokes_flow#Demonstration_of_time-reversibility

## This notebook

Notebooks are a popular way to develop and share computational processes. They are somewhere between a REPL (enter things at a command line and get the results), and a more traditional program or script. They allow for commenting (like this) and inline presentation of images (as below). They are very useful for developing ideas and algorithms, much like a physical notebook.

This notebook is quite long, to try and give some insight into how things work and the experimental process by which numerical algorithms are developed, but don't be put off by how long and code-heavy it is. Code found here could of course be adapted an encapsulated more, for futher usage. (And at the time of this writing, similar approaches in 3D are being used to develop an audiovisual installation).

(If you use Julia enough to prefer normal `.jl` code, you can also use [Jupytext](https://github.com/mwouts/jupytext) or similar, e.g. ` jupytext --to jl PRiSM_Tool.ipynb` and adapt the resulting code to your needs)

## To Run
In the "Cell" menu, select "Run All".

To re-run from scratch, click the ⏩ button.

See `traces/` for output files generated at the bottom, and `play_trace.scd` for a (very) rudimentary example of interpreting this data with SuperCollider.

Because Julia [compiles efficient low-level code "Just in Time"](https://docs.julialang.org/en/v1/#man-introduction), the first time running this notebook will be slower than subsequent runs.

## Parameters
Some of the more "twiddle-able" parameters are collected here.

In [ ]:
image_name = "images/scribble.png"

max_level = 7;

n_traces = 7
x0s = LinRange(0.1, 0.9, n_traces);

## Activate Package
This takes care of all the dependencies (other pieces of software) that we rely on.
The first time you run it, this will require an internet connection and may take a long time!

In [ ]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate()

In [ ]:
using Revise  # Development tool - must come first

In [ ]:
using MPI  # Message Passing Interface (used by p4est, even though we run serially, here)
MPI.Init();  # If familiar with MPI, you may want to omit this and/or finalize later

## Custom Mesh Package
This package provides a simple wrapper for some of the functionality of [p4est](https://p4est.org), a powerful, low-level library for massively-parallel adaptive mesh refinement (AMR). It's also [available on GitHub](https://github.com/psanan/SimpleTreeMeshes.jl).

In [ ]:
using SimpleTreeMeshes

## Helpers
Other packages to help with plotting, printing, etc.

In [ ]:
using Plots
using Printf
using Images, FileIO

## An image used to drive the simulation
If you want to refine a lot, make sure it's high-enough resolution!

In [ ]:
img = Gray.(load(image_name))
sz = size(img)
mask_size = min(sz[1], sz[2])
img2 = img[1:mask_size, 1:mask_size]
mask_r = convert(Array{Float16}, img2)
mask = zeros(size(mask_r))
for i = 1:mask_size
    for j = 1:mask_size
        mask[i,j] = mask_r[mask_size - j + 1,i]
    end
end
display(img2)

## Defining a Physical Problem

In [ ]:
struct Problem
  name::String
  boundary_conditions::StokesBoundaryConditions
  eta::Function
  eta_min::Float64
  eta_max::Float64
  eta_ref::Float64
  f_x::Function
  f_y::Function
  f_p::Function
  vx_ref::Function
  vy_ref::Function
  p_ref::Function
  coordinate_scale::Float64
  coordinate_offset_x::Float64
  coordinate_offset_y::Float64
end

In [ ]:
function image_value(x, y)
  @assert 0 <= x <= 1  
  @assert 0 <= y <= 1  
  return mask[Int(floor(x * mask_size)), Int(floor(y * mask_size))]
end;

function problem_Image()
    eta(x, y) = 1.0
    eta_min = 1.0
    eta_max= 1.0
    eta_ref = sqrt(eta_min * eta_max)

    f_vy(x,y) = -10.0 * (1.0 - image_value(x,y))
    f_vx(x, y) = 0.0
    f_p(x, y) = 0.0 
    
    vx_ref(x, y) = 0.0
    vy_ref(x, y) = 0.0
    p_ref(x, y) = 0.0
 
    return Problem("Image", stokes_boundary_free_slip, eta, eta_min, eta_max, eta_ref, f_vx, f_vy, f_p, 
                   vx_ref, vy_ref, p_ref, 1.0, 0.0, 0.0)
    
end;

## Refinement function

In [ ]:
min_level = 1;

function refinement_function(x, y, d, level)::Bool
    
    if level <= min_level;  return true; end
    if level >= max_level;  return false;  end

    # attempt to refine if all local pixels aren't in a constant band of values 
    mx_min = max(1, Int(ceil(x * (mask_size - 1))))
    my_min = max(1, Int(ceil(y * (mask_size - 1))))

    mx_max = max(1, Int(ceil((x + d) * (mask_size - 1))))
    my_max = max(1, Int(ceil((y + d) * (mask_size - 1))))

    max_local = 0
    min_local = 1.0

    for i = mx_min:mx_max
        for j = my_min:my_max
            val = mask[i,j]
            if val > max_local
                max_local = val
            end
            if val < min_local
                min_local = val
            end
        end
    end

    return max_local - min_local > 0.2
end;

## Create our tree object

In [ ]:
start = time()
tree = CreateSimpleTreeMesh(refinement_function);
@printf "⚙️ %d elements, %d faces, %d corners " tree.ne tree.nf tree.nc
@printf "⏲️ %2.2f s\n" time()- start

## Plot to see the grid

In [ ]:
start = time()
plot_size = 1000
p_grid = plot(axis=nothing, xaxis=false, yaxis=false, size=(plot_size, plot_size), aspect_ratio=1)
plot_grid!(p_grid, tree, RGB(0.5, 0.5, 0.5))
plot_face_numbers!(p_grid, tree)
plot_element_numbers!(p_grid, tree, offset=tree.nf) # Number after faces, as in our Stokes system
plot_corner_numbers!(p_grid, tree) # A separate numbering system
display(p_grid)
@printf "⏲️ %2.2f s\n" time()- start

## Solve the Stokes system

In [ ]:
start = time()
h_min = 2.0^(-(max_level))
problem = problem_Image()
A, b, Kcont = assemble_system(tree, problem, h_min)
x = A\b
v, p = sol2vp(tree, x, Kcont)
@printf "⏲️ %2.2f s\n" time() - start

In [ ]:
start = time()
plot_size = 1000
p_stokes = plot(axis=nothing, xaxis=false, yaxis=false, size=(plot_size, plot_size), aspect_ratio=1)
plot_element_field!(p_stokes, tree, p)
c = RGB(0.25, 0.25, 0.25)
plot_grid!(p_stokes, tree, c)
plot_averaged_velocity_field!(p_stokes, tree, v, color=c)
display(p_stokes)
@printf "⏲️ %2.2f s\n" time() - start

## Push particles and generate traces
Using a couple of helper functions to locate points and extract velocity,
Extract some data on the paths of particles in the fluid.

In [ ]:
struct Trace
  id::Int # An integer identifier
  x::Array{Float64}  
  y::Array{Float64}  
  t::Array{Float64}
  p::Array{Float64} 
end;

In [ ]:
function create_trace(tree::SimpleTreeMesh, x0, y0, v, p, dt=300.0, nsteps=200; id=0)
    @assert length(v) == tree.nf
    @assert length(p) == tree.ne
    trace = Trace(id, [], [], [], [])
    e_id = tree.ne ÷ 2
    particle_y = y0
    particle_x = x0
    t = 0
    dt = 1.0
    for step in 1:nsteps
    
       # Collect data from the current position
       append!(trace.x, particle_x)
       append!(trace.y, particle_y)
       append!(trace.t, t)
       e_id = locate_point(tree, particle_x, particle_y, e_id)
       append!(trace.p, p[e_id]) 
        
       # An extremely naive (forward Euler) particle position update
       vx, vy = get_point_velocity(tree, particle_x, particle_y, e_id, v)
       particle_x += dt * vx
       particle_y += dt * vy
        
       if particle_x < tree.coordinate_offset_x; particle_x = tree.coordinate_offset_x; end
       if particle_x > tree.coordinate_offset_x + tree.coordinate_scale; particle_x = tree.coordinate_offset_x + tree.coordinate_scale; end
       if particle_y < tree.coordinate_offset_y; particle_y = tree.coordinate_offset_y; end
       if particle_y > tree.coordinate_offset_y + tree.coordinate_scale; particle_y = tree.coordinate_offset_y + tree.coordinate_scale; end
       t += dt
    end
    return trace
end;

In [ ]:
start = time()
traces = []
for i = 1:length(x0s)
  x0 = x0s[i] 
  y0 = x0
  append!(traces, [create_trace(tree, x0, y0, v, p, id=i)])
end
@printf "⏲️ %2.2f s\n" time() - start

## Plot Traces

Add traces on the existing plot and re-display:

In [ ]:
start = time()
for trace in traces
    plot!(p_stokes, trace.x, trace.y, markershape=:circle, label=nothing, color=trace.id)
    plot!(p_stokes, [trace.x[1]], [trace.y[1]], markershape=:star, markersize=10, label=nothing, color=trace.id)
end
display(p_stokes)
@printf "⏲️ %2.2f s\n" time() - start

Plot pressure vs. time for the traces:

In [ ]:
start = time()
p_traces = plot(xlabel="t", ylabel="p", size=(800,600))
for trace in traces
    label = @sprintf "Trace %d" trace.id
    plot!(p_traces, trace.t, trace.p, marker=:circle, label=label, color=trace.id)
    plot!(p_traces, [trace.t[1]], [trace.p[1]], markershape=:star, markersize=10, label=nothing, color=trace.id)
end
display(p_traces)
@printf "⏲️ %2.2f s\n" time() - start

## Write traces to files

In [ ]:
start = time()
if ~isdir("traces"); mkdir("traces"); end
for (trace_id, trace) in enumerate(traces)
    filename = @sprintf "traces/trace_%04d.txt" trace_id
    open(filename, "w") do io
        for i = 1:length(trace.t)
            line = @sprintf "%g %g %g %g\n" trace.t[i] trace.p[i] trace.x[i] trace.y[i]
            write(io, line)
        end
    end
end;
@printf "⏲️ %2.2f s\n" time() - start